In [43]:
import pandas as pd
import re

In [44]:
# 파일 불러오기
customer_df_original = pd.read_csv('export_customer_20240314_140305.csv')
business_df_original = pd.read_excel('사업자DB.xlsx')

In [45]:
customer_df_original.tail()

,email,_website,_store,website_id,store_id,created_in,prefix,firstname,middlename,lastname,...,amrewards_earning_notification,amrewards_expire_notification,mobile_number,sms_agree,email_agree,vendor_code,vendor_name,admin_memo,show_password,password
2463,01074054992@dalratmart.com,base,th,1,1,TH,NaN,시롬타이,NaN,S23107_R_사,...,NaN,NaN,1074054992,Yes,Yes,NaN,NaN,suradech 사업자 아이디,ZrCpr0g3,NaN
2464,01080883445@dalratmart.com,base,th,1,1,TH,NaN,Asia mart Manseon,NaN,S23011_F_사,...,NaN,NaN,1080883445,Yes,No,NaN,NaN,NaN,HtWUBTW1,NaN
2465,01049952314@dalratmart.com,base,th,1,1,TH,NaN,란나무까타 의정부,NaN,S24033_F_사,...,NaN,NaN,1049952314,Yes,Yes,NaN,NaN,NaN,oF4vMZxY,NaN
2466,023185666@dalratmart.com,base,th,1,1,TH,NaN,바나나리프,NaN,S22118_T_사,...,NaN,NaN,23185666,Yes,Yes,NaN,NaN,*(S22118_T)바나나리프 아로이타이수원 사업자 아이디,G3KtWnjU,NaN
2467,01054322861@dalratmart.com,base,th,1,1,TH,NaN,넝지나아시아마트,NaN,S24042_F_사,...,NaN,NaN,1054322861,Yes,No,NaN,NaN,NaN,6ukbuHAP,NaN


In [46]:
business_df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   아이디       286 non-null    object
 1   휴대폰 번호    282 non-null    object
 2   대리점아이디    286 non-null    object
 3   사업자 등록번호  286 non-null    object
 4   사업자명      286 non-null    object
 5   성명        286 non-null    object
 6   주소        286 non-null    object
 7   업태        285 non-null    object
 8   종목        285 non-null    object
 9   이메일       84 non-null     object
 10  비고        5 non-null      object
dtypes: object(11)
memory usage: 24.7+ KB


In [47]:
# 고객 데이터에서 'mobile_number' 처리
customer_df_original['mobile_number'] = customer_df_original['mobile_number'].astype(str)
customer_df_original['mobile_number'] = customer_df_original['mobile_number'].apply(lambda x: '0' + x if not x.startswith('0') else x)
customer_df_original['mobile_number'] = customer_df_original['mobile_number'].str.extract('(\d+)', expand=False)

# 사업자 데이터에서 '휴대폰 번호' 처리
business_df_original['휴대폰 번호'] = business_df_original['휴대폰 번호'].astype(str)
business_df_original['휴대폰 번호'] = business_df_original['휴대폰 번호'].str.replace(r'\D', '', regex=True)

# 일치하는 휴대폰 번호 찾아 '사업자' 컬럼 추가
customer_df_original['사업자'] = customer_df_original['mobile_number'].isin(business_df_original['휴대폰 번호']).apply(lambda x: '1_' if x else '')

In [48]:
customer_df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2468 entries, 0 to 2467
Data columns (total 38 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   email                           2468 non-null   object 
 1   _website                        2468 non-null   object 
 2   _store                          2468 non-null   object 
 3   website_id                      2468 non-null   int64  
 4   store_id                        2468 non-null   int64  
 5   created_in                      2468 non-null   object 
 6   prefix                          0 non-null      float64
 7   firstname                       2468 non-null   object 
 8   middlename                      0 non-null      float64
 9   lastname                        2468 non-null   object 
 10  suffix                          0 non-null      float64
 11  group_id                        2468 non-null   int64  
 12  dob                             2 

In [49]:
df = customer_df_original

In [50]:
# 1. 'S+숫자5자리' 부분 삭제
df['firstname'] = df['firstname'].str.replace('S\d{5}', '', regex=True)
df['lastname'] = df['lastname'].str.replace('S\d{5}', '', regex=True)

In [51]:
df['firstname'][20:30]

20    _T KHUNAROOP
21           Seeya
22     PATAMAWADEE
23          Anucha
24         Navapat
25        Sarintip
26        Sonnarin
27       Piyavadee
28         Sitisuk
29        Suphawan
Name: firstname, dtype: object

In [52]:
# 2. 중복되는 값 제거 후 'firstname' 업데이트
df['firstname'] = df.apply(lambda x: x['firstname'] if x['firstname'] == x['lastname'] else x['firstname'] + ' ' + x['lastname'], axis=1).str.strip()

In [53]:
# 3. 'lastname'컬럼비우기
df['lastname'] = ''

In [54]:
# 4. 'operator' 값을 기준으로 'lastname' 업데이트
operator_mappings = {
    '돈철 이': '_N',
    '팁 팁': '_T',
    '파이 파이': '_F',
    '종형 변': '_H',
    '룽 룽': '_R'
}
df['lastname'] = df['operator'].map(operator_mappings).fillna('')

In [55]:
df

,email,_website,_store,website_id,store_id,created_in,prefix,firstname,middlename,lastname,...,amrewards_expire_notification,mobile_number,sms_agree,email_agree,vendor_code,vendor_name,admin_memo,show_password,password,사업자
0,testcustomer@test.com,base,th,1,1,Dalra B2B Thailand Store View,NaN,Customer Test,NaN,_N,...,NaN,01011111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,billynguyen891@gmail.com,base,th,1,1,Dalra B2B Thailand Store View,NaN,Billy,NaN,_N,...,NaN,0123321123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,billytest@gmail.com,base,th,1,1,Dalra B2B Thailand Store View,NaN,Dev,NaN,_N,...,NaN,01011111112,No,No,NaN,NaN,NaN,NaN,NaN,
3,test777@test.com,base,th,1,1,Dalra B2B Thailand Store View,NaN,Test,NaN,_N,...,NaN,0107777777,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,newtest@newtest.com,base,th,1,1,Dalra B2B Thailand Store View,NaN,new test,NaN,_N,...,NaN,0101111123,NaN,NaN,NaN,NaN,NaN,jsIEtJ8F,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463,01074054992@dalratmart.com,base,th,1,1,TH,NaN,시롬타이 _R_사,NaN,_R,...,NaN,01074054992,Yes,Yes,NaN,NaN,suradech 사업자 아이디,ZrCpr0g3,NaN,1_
2464,01080883445@dalratmart.com,base,th,1,1,TH,NaN,Asia mart Manseon _F_사,NaN,_F,...,NaN,01080883445,Yes,No,NaN,NaN,NaN,HtWUBTW1,NaN,1_
2465,01049952314@dalratmart.com,base,th,1,1,TH,NaN,란나무까타 의정부 _F_사,NaN,_F,...,NaN,01049952314,Yes,Yes,NaN,NaN,NaN,oF4vMZxY,NaN,1_
2466,023185666@dalratmart.com,base,th,1,1,TH,NaN,바나나리프 _T_사,NaN,_R,...,NaN,023185666,Yes,Yes,NaN,NaN,*(S22118_T)바나나리프 아로이타이수원 사업자 아이디,G3KtWnjU,NaN,1_


In [56]:
# 5. 사업자 lastname에 업데이트
df['firstname'] = df['사업자'].apply(lambda x: '1_' if x == '1_' else '') + df['firstname']

In [60]:
df.drop('사업자', axis=1, inplace=True) # 사업자 컬럼제거

In [61]:
df

,email,_website,_store,website_id,store_id,created_in,prefix,firstname,middlename,lastname,...,amrewards_earning_notification,amrewards_expire_notification,mobile_number,sms_agree,email_agree,vendor_code,vendor_name,admin_memo,show_password,password
0,testcustomer@test.com,base,th,1,1,Dalra B2B Thailand Store View,NaN,Customer Test,NaN,_N,...,NaN,NaN,01011111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,billynguyen891@gmail.com,base,th,1,1,Dalra B2B Thailand Store View,NaN,Billy,NaN,_N,...,NaN,NaN,0123321123,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,billytest@gmail.com,base,th,1,1,Dalra B2B Thailand Store View,NaN,Dev,NaN,_N,...,NaN,NaN,01011111112,No,No,NaN,NaN,NaN,NaN,NaN
3,test777@test.com,base,th,1,1,Dalra B2B Thailand Store View,NaN,Test,NaN,_N,...,NaN,NaN,0107777777,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,newtest@newtest.com,base,th,1,1,Dalra B2B Thailand Store View,NaN,new test,NaN,_N,...,NaN,NaN,0101111123,NaN,NaN,NaN,NaN,NaN,jsIEtJ8F,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463,01074054992@dalratmart.com,base,th,1,1,TH,NaN,1_시롬타이 _R_사,NaN,_R,...,NaN,NaN,01074054992,Yes,Yes,NaN,NaN,suradech 사업자 아이디,ZrCpr0g3,NaN
2464,01080883445@dalratmart.com,base,th,1,1,TH,NaN,1_Asia mart Manseon _F_사,NaN,_F,...,NaN,NaN,01080883445,Yes,No,NaN,NaN,NaN,HtWUBTW1,NaN
2465,01049952314@dalratmart.com,base,th,1,1,TH,NaN,1_란나무까타 의정부 _F_사,NaN,_F,...,NaN,NaN,01049952314,Yes,Yes,NaN,NaN,NaN,oF4vMZxY,NaN
2466,023185666@dalratmart.com,base,th,1,1,TH,NaN,1_바나나리프 _T_사,NaN,_R,...,NaN,NaN,023185666,Yes,Yes,NaN,NaN,*(S22118_T)바나나리프 아로이타이수원 사업자 아이디,G3KtWnjU,NaN


In [58]:
# 결과 확인 및 파일 다시 저장
output_path = '고객최종수정본.xlsx' #저장 이름 및 경로 지정
df.to_excel(output_path, index=False) #인덱스 포함하지 않고 저장